# Train Model with XLA JIT Enabled

## Run the Next Cell to Show XLA JIT Training Code

In [ ]:
cat /root/src/main/python/xla/train_linear_xla.py

## Run the Next Cell to Train with XLA Enabled

The cell below will report 
```   
Aborted (core dumped)
```
but still generates useful info.

In [ ]:
%%bash

train_xla

### Click on generated files in File Browser in Left Navigation
* `train_linear_xla.log` **(Log File)**  
* `hlo_graph_*` **(JIT Visualizations)** 

## Reference to All XLA Operations
https://www.tensorflow.org/performance/xla/operation_semantics